# Capstone Project Report – The Battle of Neighbourhoods in Chicago

## Introduction/Business Problem

A sushi franchise owner is seeking perfect locations to open branches where he can intorduce the finest sushi to the residents of the city. However, he is new to the city and couldn't decide where to set root for the business to grow. The three rules for starting a business are 1)location, 2)location, and 3)location! Therefore, he seeks help from data scientists and engineers to solve the problem that could be the deciding factor to this expansion.

## Data

Retriving postal and geolocation data from wikipedia for locating the neighbourhoods in Chicago.
Use Foursquare API to explore venues in the neighbourhoods for analysis.
Use Foursquare API to search for sushi restaurants in the neighbourhoods.
Use Foursquare API to extract number of users who have liked the restaurants.

In [66]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [67]:
url = 'https://en.wikipedia.org/wiki/Community_areas_in_Chicago'
data = requests.get(url).text # send GET request and store as text data
my_soup = BeautifulSoup(data, 'html5lib') # parse the data with beautifulsoup

# search for target table
tables = my_soup.find_all('table')

for index, table in enumerate(tables):
    if 'Chicago community areas by number, population, and area' in str(table): # find the string in our target table that is unique to the other tables
        target_table_index = index
print('There are {} tables found.\nTarget Table Index : {}'.format(index + 1, target_table_index))

There are 4 tables found.
Target Table Index : 0


In [68]:
community_df =  pd.DataFrame(columns = ['No.', 'Name', 'Latitude', 'Longitude'])

for count, row in enumerate(tables[target_table_index].tbody.find_all('tr')):
    if count > 1:
        number = row.find('td').getText().replace('\n', '')
        name = row.find('a').getText().replace('\n', '')
        community_df = community_df.append({'No.' : number, 'Name' : name}, ignore_index = True)

community_df.drop([count - 2], inplace = True)
community_df

,No.,Name,Latitude,Longitude
0,01,Rogers Park,NaN,NaN
1,02,West Ridge,NaN,NaN
2,03,Uptown,NaN,NaN
3,04,Lincoln Square,NaN,NaN
4,05,North Center,NaN,NaN
...,...,...,...,...
72,73,Washington Heights,NaN,NaN
73,74,Mount Greenwood,NaN,NaN
74,75,Morgan Park,NaN,NaN
75,76,O'Hare,NaN,NaN
